## Dataset statistics

In [ ]:



from func import *
import os
import pandas as pd
import numpy as np


def make_sens_vector_(df, dataset, sens_dict):
    if dataset in ['Walmart-Amazon', 'Fodors-Zagat', 'COMPAS']:
        df['left_contains_s'] = df[sens_dict[dataset][0]] == sens_dict[dataset][1]
        
    elif dataset in ['Beer', 'Amazon-Google', 'iTunes-Amazon', 'DBLP-GoogleScholar']:
        df[sens_dict[dataset][0]] = df[sens_dict[dataset][0]].astype(str)
        df['left_contains_s'] = df[sens_dict[dataset][0]].str.lower().str.contains(sens_dict[dataset][1].lower())
    else:
        # df[sens_dict[dataset][0]] = df[sens_dict[dataset][0]].astype(str)
        # df['left_contains_s'] = df[sens_dict[dataset][0]].apply(lambda x: x.replace('&#216;', '').replace('&#214;', '').replace('&#237;', ',').split(',')[-1].strip())
        # df['left_contains_s'] = df['left_contains_s'].apply(lambda x: ', '.join([gender_rev(name) for name in x.split(',')]))
        # df['left_contains_s'] = df['left_contains_s'].apply(lambda x: 'True' if 'female' in str(x) else 'False')
        # df['left_contains_s'] = df['left_contains_s'].apply(lambda x: any(item in x for item in ['True']))


        df[sens_dict[dataset][0]] = df[sens_dict[dataset][0]].astype(str)
        df['left_contains_s'] = df[sens_dict[dataset][0]].apply(lambda x: x.replace('&#216;', '').replace('&#214;', '').replace('&#237;', ',').split(',')[-1].strip())


        sens = [] 
        for x  in list(df['left_contains_s']):
            try: 
                tmp = str(gender_rev(x))
            except:
                tmp = '?'
            if 'female' in tmp:
                GENDER = True
            else:
                GENDER = False
            sens.append(GENDER)

        df['left_contains_s'] = sens
        result_vector = np.array(df['left_contains_s'])
        sens_attr = np.array(result_vector).reshape(-1)
        return sens_attr

    result_vector = np.array(df['left_contains_s']).astype(int)
    sens_attr = np.array(result_vector).reshape(-1)

    return sens_attr



for path in os.listdir('data/'):
    if 'DS_Store' in path: continue
    
    tableA = pd.read_csv('data/' + path  +'/tableA.csv')
    tableB = pd.read_csv('data/' + path  +'/tableB.csv')
    golden = pd.read_csv('data/' + path  +'/matches.csv')
    
    print(path, tableA.shape[0], tableB.shape[0], round((tableA.shape[0] * tableB.shape[0])/1000,1), golden.shape[0])

print()

for task in [
    'Walmart-Amazon',
                'Beer',
                'Amazon-Google',
                'Fodors-Zagat',
                'iTunes-Amazon',
                'DBLP-GoogleScholar',
                'Febrl',
                'DBLP-ACM'
                ]:




    
    folder_root = "data/" + task
    left_df = pd.read_csv(folder_root +'/tableA.csv')
    right_df = pd.read_csv(folder_root +'/tableB.csv')
    match_df = pd.read_csv(folder_root +'/matches.csv')
    match_df.rename(columns={list(match_df.columns)[0]: 'ltable_id', list(match_df.columns)[1]: 'rtable_id'}, inplace=True)



    left_sens = make_sens_vector_(left_df.copy(), task, sens_dict)
    right_sens = make_sens_vector_(right_df.copy(), task, sens_dict)

    
    df_A = pd.DataFrame(left_sens.astype(bool))
    df_B = pd.DataFrame(right_sens.astype(bool))

    match_sens = make_candid_sens(match_df, left_df.copy(), right_df.copy(),task, sens_dict)


    print(task , 'A',np.sum(df_A)[0], 'B',np.sum(df_B)[0], np.sum(match_sens['sensitive']))
    # break


##  Blocking stats, Bias



In [ ]:
import os
import time
import copy
from func import *
import numpy as np
import pandas as pd

from pyjedai.datamodel import Data
from pyjedai.block_cleaning import BlockPurging, BlockFiltering
from pyjedai.block_building import (
    StandardBlocking,
    ExtendedQGramsBlocking,
    ExtendedSuffixArraysBlocking,
    QGramsBlocking,
    SuffixArraysBlocking
)
from pyjedai.comparison_cleaning import (
    WeightedEdgePruning, WeightedNodePruning, 
    CardinalityEdgePruning, CardinalityNodePruning,
    BLAST, ReciprocalCardinalityNodePruning,
    ReciprocalWeightedNodePruning, ComparisonPropagation)


# Define method dictionaries
classic_method_dict = {
                'SB': StandardBlocking(),
                'QG': QGramsBlocking(),
                'EQG': ExtendedQGramsBlocking(),
                'SA': SuffixArraysBlocking(),
                'ESA': ExtendedSuffixArraysBlocking()
                }
classic_method_name = {
    'SB': 'StandardBlocking',
    'EQG': 'ExtendedQGramsBlocking',
    'ESA': 'ExtendedSuffixArraysBlocking',
    'QG': 'QGramsBlocking',
    'SA': 'SuffixArraysBlocking',
    'CTT': 'CTT',
    'AE': 'AUTO',

}

output_file = "block_stat.txt"
# output_file = "tmp.txt"

# Remove output file if it exists
try: os.remove(output_file)
except FileNotFoundError: pass

# Define tasks and methods to iterate over
tasks = ['Beer', 'Walmart-Amazon', 'Amazon-Google', 'DBLP-ACM', 'Fodors-Zagat', 'DBLP-GoogleScholar']# 'iTunes-Amazon'
methods = ['SB', 'EQG', 'ESA', 'QG','SA','CTT','AE']



# Process each task with each method
for task in tasks:
    for method in methods:
        
        # Load datasets
        left_df = pd.read_csv(f"data/{task}/tableA.csv")
        right_df = pd.read_csv(f"data/{task}/tableB.csv")
        match_df = pd.read_csv(f"data/{task}/matches.csv")

        # Clean datasets for specific tasks
        if task == 'Fodors-Zagat':
            for df in [left_df, right_df]:
                df.applymap(lambda x: x.strip('`').strip() if isinstance(x, str) else x)
                df.applymap(lambda x: x.strip("'").strip() if isinstance(x, str) else x)

        left_df = left_df.replace(r"\\ '", "'", regex=True).replace(r" '", "'", regex=True).replace(r"\\ `", "\\ ", regex=True)
        right_df = right_df.replace(r"\\ '", "'", regex=True).replace(r" '", "'", regex=True).replace(r"\\ `", "\\ ", regex=True)
        
        left_df = left_df.applymap(lambda x: x.strip('`') if isinstance(x, str) else x).applymap(lambda x: x.strip("'") if isinstance(x, str) else x).applymap(lambda x: x.strip() if isinstance(x, str) else x)
        right_df = right_df.applymap(lambda x: x.strip('`') if isinstance(x, str) else x).applymap(lambda x: x.strip("'") if isinstance(x, str) else x).applymap(lambda x: x.strip() if isinstance(x, str) else x)



        # Process using classic method if applicable
        if method in classic_method_dict:
            bb = classic_method_dict[method]
            attr = [col for col in left_df.columns if col != 'id']
            data = Data(
                dataset_1=left_df.copy(), id_column_name_1='id',
                dataset_2=right_df.copy(), id_column_name_2='id',
                ground_truth=match_df.rename(columns={list(match_df.columns)[0]: 'D1', list(match_df.columns)[1]: 'D2'})
            )
            data.clean_dataset(remove_stopwords=False, remove_punctuation=False, remove_numbers=False, remove_unicodes=True)
            
            start_time = time.time()
            blocks = bb.build_blocks(copy.deepcopy(data), attributes_1=attr, attributes_2=attr, tqdm_disable=True)
            
            bp = BlockPurging()
            bf = BlockFiltering()
            mb = BLAST('EJS')
            if task == 'iTunes-Amazon':
                mb = CardinalityEdgePruning()
            # for meta in META:
            cleaned_blocks = bf.process(copy.deepcopy(blocks), data, tqdm_disable=True)
            filtered_blocks = bp.process(cleaned_blocks, data, tqdm_disable=True)
            candidate_pairs_blocks = mb.process(filtered_blocks, data, tqdm_disable=False)

            candidates = mb.export_to_df(candidate_pairs_blocks)
        
        else:
            if method == 'CTT':
                start_time = time.time()
                candidates = deepBlock(left_df.copy(), right_df.copy(), K = 50, method = 'CTT')
            elif method =='AE':
                start_time = time.time()
                candidates = deepBlock(left_df.copy(), right_df.copy(), K = 50, method = 'AE')
            else:
                print('method not found!')

        end_time = time.time()

        # Compile result rows
        candidates= candidates.astype(int)

        # Rename columns in candidates DataFrame
        candidates.rename(columns={'ltable_id': 'id1', 'rtable_id': 'id2'}, inplace=True)

        # Merge left_df and right_df with candidates based on ids using vectorized operations
        left_merged = candidates.merge(left_df, left_on='id1', right_on='id', suffixes=('', '_left'))
        right_merged = left_merged.merge(right_df, left_on='id2', right_on='id', suffixes=('_left', '_right'))

        # Drop redundant columns and reset index
        result_df = right_merged.drop(columns=['id1', 'id2', 'id_left', 'id_right'])
        result_df = right_merged.copy()

        # Merge with match_df to determine labels
        merged_df = result_df.merge(match_df, left_on=['id_left', 'id_right'], right_on=['ltable_id', 'rtable_id'], how='left', indicator=True)
        result_df['label'] = (merged_df['_merge'] == 'both').astype(int)
        # result_df.to_csv(task +'_'+method+ '.csv',index= False)

        # Calculate metrics
        RR = 1 - result_df.shape[0] / (left_df.shape[0] * right_df.shape[0])
        RR_opt = 1 - match_df.shape[0] / (left_df.shape[0] * right_df.shape[0])
        PC = np.sum(result_df['label'] == 1) / match_df.shape[0]
        PQ = np.sum(result_df['label'] == 1) / result_df.shape[0]
        F = 2 * PC * PQ / (PC + PQ)

        # Print and save results
        print(task, classic_method_name[method])
        print(round(100 * RR, 4), round(100 * PC, 4), round(100 * PQ, 4), round(100 * F, 4))
        print()

        runtime = end_time - start_time



        MINOR, MAJOR, data_STAT = calc_bias_block(result_df,match_df,left_df,right_df, task , sens_dict)
        [RR_minor,PC_minor,PQ_minor,Fb_minor ] = MINOR
        [RR_major,PC_major,PQ_major,Fb_major ] = MAJOR
        [P_major, P_minor, M_major, M_minor] = data_STAT

        NUM = 2
        print('Minor: ',end='')
        print(round(RR_minor,NUM),round(PC_minor,NUM),round(PQ_minor,NUM), round(Fb_minor,NUM))
        print('major: ',end='')
        print(round(RR_major,NUM),round(PC_major,NUM),round(PQ_major,NUM), round(Fb_major,NUM))
        print('diff : ',end='')
        print(round(RR_major-RR_minor ,NUM),round(PC_major- PC_minor,NUM),round(PQ_major - PQ_minor,NUM),round(Fb_major - Fb_minor,NUM))


        print()
        with open(output_file, "a") as file:
            file.write(f"{task} {classic_method_name[method]}\n")
            file.write(f"{round(100 * RR, 4)} {round(100 * PC, 4)} {round(100 * PQ, 4)} {round(100 * F, 4)} {round(runtime, 2)}\n\n")
            file.write(f"bias {round( RR_minor, 4)} {round( PC_minor, 4)} {round( PQ_minor, 4)} {round( Fb_minor, 4)}\n\n")
            file.write(f"bias {round( RR_major, 4)} {round( PC_major, 4)} {round( PQ_major, 4)} {round( Fb_major, 4)}\n\n")
            file.write(f"bias {round( RR_major - RR_minor, 4)} {round( PC_major - PC_minor, 4)} {round( PQ_major - PQ_minor, 4)} {round( Fb_major - Fb_minor, 4)}\n\n")
            file.write(f"bias {P_major} {P_minor} {M_major} {M_minor}\n\n")






## convert txt to latex table result

In [ ]:
import pandas as pd

# Define the path to your text file
file_path = 'block_stat.txt'

# Read the file and process the data
data = []
res = {}
with open(file_path, 'r') as file:
    for line in file:
        # Split the line into parts based on spaces
        parts = line.split()
        if parts == []: continue
        if len(parts) == 2:
            if parts[0] not in res.keys():
                res[parts[0]] = {}
            res[parts[0]][parts[1]] = {}
            method = parts[1]
            dataset = parts[0]
        else:
            RR = float(parts[0])
            PC = float(parts[1])
            PQ = float(parts[2])
            time_ = float(parts[4])
            Fb = 2*PC *RR / (PC + RR)
            res[dataset][method] = {'RR':round(RR,5), 'PC':round(PC,5), 'PQ':round(PQ,5), 'Fb':round(Fb,5), 'time':round(time_,5)}


METHODS = {
    'StandardBlocking':'\\stdBlock',
    'QGramsBlocking':'\\qgram',
    'ExtendedQGramsBlocking':'\\exQgram',
    'SuffixArraysBlocking':'\\suffix',
    'ExtendedSuffixArraysBlocking':'\\exSuffix',
    'AUTO':'\\AutoBlock',
    'CTT':'\\CTT'}

# TASKS = ['Amazon-Google','Walmart-Amazon','DBLP-GoogleScholar', 'DBLP-ACM','Beer','Fodors-Zagat','iTunes-Amazon']
# for method in METHODS.keys():
#     for task in TASKS:
    
#         row = res[task][method]
#         if task == 'iTunes-Amazon':
#             print(f"{row['RR']:.2f} & {row['PC']:.2f} & {row['PQ']:.2f} & {row['Fb']:.2f} \\\\")
#         elif task == 'Amazon-Google':
#             print(f"{METHODS[method]} & \n{row['RR']:.2f} & {row['PC']:.2f} & {row['PQ']:.2f} & {row['Fb']:.2f} & ")
#         else:
#             print(f"{row['RR']:.2f} & {row['PC']:.2f} & {row['PQ']:.2f} & {row['Fb']:.2f} & ")


#     print()


TASKS = ['Amazon-Google', 'Walmart-Amazon', 'DBLP-GoogleScholar', 'DBLP-ACM', 'Beer', 'Fodors-Zagat', 'iTunes-Amazon']

# Open a file in write mode
with open('exp1_latex.txt', 'w') as file:
    for method in METHODS.keys():
        for task in TASKS:
            row = res[task][method]
            if task == 'iTunes-Amazon':
                file.write(f"{row['RR']:.2f} & {row['PC']:.2f} & {row['PQ']:.2f} & {row['Fb']:.2f} \\\\\n")
            elif task == 'Amazon-Google':
                file.write(f"{METHODS[method]} & \n{row['RR']:.2f} & {row['PC']:.2f} & {row['PQ']:.2f} & {row['Fb']:.2f} & \n")
            else:
                file.write(f"{row['RR']:.2f} & {row['PC']:.2f} & {row['PQ']:.2f} & {row['Fb']:.2f} & \n")

        # Add a newline after each method
        file.write('\n')


## correlation of RR, PC, PQ

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Initialize the overall correlation matrix
correlation_matrix_all = np.zeros((3, 3))

# Iterate over tasks
for i in range(7):
    RR = []
    PC = []
    PQ = []
    
    # Calculate metrics for each method
    for method in METHODS.keys():
        task = TASKS[i]
        row = res[task][method]
        RR.append(row['RR'])
        PC.append(row['PC'])
        PQ.append(row['PQ'])

    # Convert metrics to numpy arrays
    rr = np.array(RR)
    pc = np.array(PC)
    pq = np.array(PQ)

    # Create a DataFrame and compute the correlation matrix
    data = pd.DataFrame({'RR': rr, 'PC': pc, 'PQ': pq})
    correlation_matrix_all += np.abs(data.corr())

# Plot the overall correlation heatmap
plt.figure(figsize=(10, 10))
colorblind_palette = sns.color_palette("colorblind", as_cmap=True)

sns.heatmap(
    (correlation_matrix_all / 7), 
    annot=True, 
    cmap=colorblind_palette, 
    center=0, 
    annot_kws={"size": 40, "weight": "bold"},
    cbar=False
)

plt.xticks(fontsize=60)
plt.yticks(fontsize=60)
plt.tick_params(axis='both', which='both', length=10, width=3)
plt.tight_layout()
# plt.savefig('correlation_heatmap_all.pdf')
# plt.close()


## time experiment

In [ ]:
import os
import time
import copy
from func import *
import numpy as np
import pandas as pd

from pyjedai.datamodel import Data
from pyjedai.block_cleaning import BlockPurging, BlockFiltering
from pyjedai.block_building import (
    StandardBlocking,
    ExtendedQGramsBlocking,
    ExtendedSuffixArraysBlocking,
    QGramsBlocking,
    SuffixArraysBlocking
)
from pyjedai.comparison_cleaning import BLAST, CardinalityEdgePruning


# Define method dictionaries
classic_method_dict = {
                'SB': StandardBlocking(),
                'QG': QGramsBlocking(),
                'EQG': ExtendedQGramsBlocking(),
                'SA': SuffixArraysBlocking(),
                'ESA': ExtendedSuffixArraysBlocking()
                }
classic_method_name = {
    'SB': 'StandardBlocking',
    'EQG': 'ExtendedQGramsBlocking',
    'ESA': 'ExtendedSuffixArraysBlocking',
    'QG': 'QGramsBlocking',
    'SA': 'SuffixArraysBlocking',
    'CTT': 'CTT',
    'AE': 'AUTO',

}

output_file = "tmp.txt"
# output_file = "tmp.txt"

# Remove output file if it exists
try: os.remove(output_file)
except FileNotFoundError: pass

# Define tasks and methods to iterate over
tasks = ['iTunes-Amazon']
methods = ['SB', 'EQG', 'ESA', 'QG','SA']

# Process each task with each method
for task in tasks:
    for method in methods:
        
        # Load datasets
        left_df = pd.read_csv(f"data/{task}/tableA.csv")
        right_df = pd.read_csv(f"data/{task}/tableB.csv")
        match_df = pd.read_csv(f"data/{task}/matches.csv")

        # Clean datasets for specific tasks
        if task == 'Fodors-Zagat':
            for df in [left_df, right_df]:
                df.applymap(lambda x: x.strip('`').strip() if isinstance(x, str) else x)
                df.applymap(lambda x: x.strip("'").strip() if isinstance(x, str) else x)

        left_df.replace(r"\\ '", "'", regex=True).replace(r" '", "'", regex=True).replace(r"\\ `", "\\ ", regex=True)
        right_df.replace(r"\\ '", "'", regex=True).replace(r" '", "'", regex=True).replace(r"\\ `", "\\ ", regex=True)

        # Process using classic method if applicable
        if method in classic_method_dict:
            bb = classic_method_dict[method]
            attr = [col for col in left_df.columns if col != 'id']
            data = Data(
                dataset_1=left_df.copy(), id_column_name_1='id',
                dataset_2=right_df.copy(), id_column_name_2='id',
                ground_truth=match_df.rename(columns={list(match_df.columns)[0]: 'D1', list(match_df.columns)[1]: 'D2'})
            )
            data.clean_dataset(remove_stopwords=False, remove_punctuation=False, remove_numbers=False, remove_unicodes=True)
            
            start_time = time.time()
            blocks = bb.build_blocks(copy.deepcopy(data), attributes_1=attr, attributes_2=attr, tqdm_disable=True)
            
            bp = BlockPurging()
            bf = BlockFiltering()
            mb = BLAST('EJS')
            if task == 'iTunes-Amazon':
                mb = CardinalityEdgePruning()

            cleaned_blocks = bf.process(blocks, data, tqdm_disable=True)
            filtered_blocks = bp.process(cleaned_blocks, data, tqdm_disable=True)
            candidate_pairs_blocks = mb.process(filtered_blocks, data, tqdm_disable=False)

            candidates = mb.export_to_df(candidate_pairs_blocks)
            end_time = time.time()

        candidates = mb.export_to_df(candidate_pairs_blocks)
        # Compile result rows
        candidates= candidates.astype(int)

        # Rename columns in candidates DataFrame
        candidates.rename(columns={'ltable_id': 'id1', 'rtable_id': 'id2'}, inplace=True)

        # Merge left_df and right_df with candidates based on ids using vectorized operations
        left_merged = candidates.merge(left_df, left_on='id1', right_on='id', suffixes=('', '_left'))
        right_merged = left_merged.merge(right_df, left_on='id2', right_on='id', suffixes=('_left', '_right'))

        # Drop redundant columns and reset index
        result_df = right_merged.drop(columns=['id1', 'id2', 'id_left', 'id_right'])
        result_df = right_merged.copy()

        # Merge with match_df to determine labels
        merged_df = result_df.merge(match_df, left_on=['id_left', 'id_right'], right_on=['ltable_id', 'rtable_id'], how='left', indicator=True)
        result_df['label'] = (merged_df['_merge'] == 'both').astype(int)
        result_df.to_csv(task +'_'+method+ '.csv',index= False)

        # Calculate metrics
        RR = 1 - result_df.shape[0] / (left_df.shape[0] * right_df.shape[0])
        RR_opt = 1 - match_df.shape[0] / (left_df.shape[0] * right_df.shape[0])
        PC = np.sum(result_df['label'] == 1) / match_df.shape[0]
        PQ = np.sum(result_df['label'] == 1) / result_df.shape[0]
        F = 2 * PC * PQ / (PC + PQ)

        # Print and save results
        print(task, classic_method_name[method])
        print(round(100 * RR, 4), round(100 * PC, 4), round(100 * PQ, 4), round(100 * F, 4))
        print()

        runtime = end_time - start_time

        with open(output_file, "a") as file:
            file.write(f"{task} {classic_method_name[method]}\n")
            file.write(f"{round(100 * RR, 4)} {round(100 * PC, 4)} {round(100 * PQ, 4)} {round(100 * F, 4)} {round(runtime, 2)}\n\n")


In [9]:
import os
import time
import copy
from func import *
import numpy as np
import pandas as pd

from pyjedai.datamodel import Data
from pyjedai.block_cleaning import BlockPurging, BlockFiltering
from pyjedai.block_building import (
    StandardBlocking,
    ExtendedQGramsBlocking,
    ExtendedSuffixArraysBlocking,
    QGramsBlocking,
    SuffixArraysBlocking
)
from pyjedai.comparison_cleaning import BLAST, CardinalityEdgePruning
from pyjedai.comparison_cleaning import (
    WeightedEdgePruning, WeightedNodePruning, 
    CardinalityEdgePruning, CardinalityNodePruning,
    BLAST, ReciprocalCardinalityNodePruning,
    ReciprocalWeightedNodePruning, ComparisonPropagation)


# Define method dictionaries
classic_method_dict = {
                'SB': StandardBlocking(),
                'QG': QGramsBlocking(),
                'EQG': ExtendedQGramsBlocking(),
                'SA': SuffixArraysBlocking(),
                'ESA': ExtendedSuffixArraysBlocking()
                }
classic_method_name = {
    'SB': 'StandardBlocking',
    'EQG': 'ExtendedQGramsBlocking',
    'ESA': 'ExtendedSuffixArraysBlocking',
    'QG': 'QGramsBlocking',
    'SA': 'SuffixArraysBlocking',
    'CTT': 'CTT',
    'AE': 'AUTO',

}

output_file = "block_stat.txt"
# output_file = "tmp.txt"

# Remove output file if it exists
try: os.remove(output_file)
except FileNotFoundError: pass

# Define tasks and methods to iterate over
tasks = ['Beer', 'Walmart-Amazon', 'Amazon-Google', 'DBLP-ACM', 'Fodors-Zagat', 'iTunes-Amazon', 'DBLP-GoogleScholar', 'Febrl']
tasks = ['iTunes-Amazon']
methods = ['SB', 'EQG', 'ESA', 'QG','SA','CTT','AE']
methods = ['SB']

META = [ 
    # CardinalityEdgePruning(),

    ComparisonPropagation()
]


# Process each task with each method
for task in tasks:
    for method in methods:
        
        # Load datasets
        left_df = pd.read_csv(f"data/{task}/tableA.csv")
        right_df = pd.read_csv(f"data/{task}/tableB.csv")
        match_df = pd.read_csv(f"data/{task}/matches.csv")

        # Clean datasets for specific tasks
        if task == 'Fodors-Zagat':
            for df in [left_df, right_df]:
                df.applymap(lambda x: x.strip('`').strip() if isinstance(x, str) else x)
                df.applymap(lambda x: x.strip("'").strip() if isinstance(x, str) else x)

        left_df.replace(r"\\ '", "'", regex=True).replace(r" '", "'", regex=True).replace(r"\\ `", "\\ ", regex=True)
        right_df.replace(r"\\ '", "'", regex=True).replace(r" '", "'", regex=True).replace(r"\\ `", "\\ ", regex=True)

        # Process using classic method if applicable
        if method in classic_method_dict:
            bb = classic_method_dict[method]
            attr = [col for col in left_df.columns if col != 'id']
            data = Data(
                dataset_1=left_df.copy(), id_column_name_1='id',
                dataset_2=right_df.copy(), id_column_name_2='id',
                ground_truth=match_df.rename(columns={list(match_df.columns)[0]: 'D1', list(match_df.columns)[1]: 'D2'})
            )
            data.clean_dataset(remove_stopwords=False, remove_punctuation=False, remove_numbers=False, remove_unicodes=True)
            
            start_time = time.time()
            blocks = bb.build_blocks(copy.deepcopy(data), attributes_1=attr, attributes_2=attr, tqdm_disable=True)
            
            bp = BlockPurging()
            bf = BlockFiltering()
            mb = BLAST('EJS')
            if task == 'iTunes-Amazon':
                mb = CardinalityEdgePruning()
            for meta in META:
                mb = copy.deepcopy(meta)
                cleaned_blocks = bf.process(copy.deepcopy(blocks), data, tqdm_disable=True)
                filtered_blocks = bp.process(cleaned_blocks, data, tqdm_disable=True)
                candidate_pairs_blocks = mb.process(filtered_blocks, data, tqdm_disable=False)

                candidates = mb.export_to_df(candidate_pairs_blocks)
        

                end_time = time.time()

                # Compile result rows
                candidates= candidates.astype(int)

                # Rename columns in candidates DataFrame
                candidates.rename(columns={'ltable_id': 'id1', 'rtable_id': 'id2'}, inplace=True)

                # Merge left_df and right_df with candidates based on ids using vectorized operations
                left_merged = candidates.merge(left_df, left_on='id1', right_on='id', suffixes=('', '_left'))
                right_merged = left_merged.merge(right_df, left_on='id2', right_on='id', suffixes=('_left', '_right'))

                # Drop redundant columns and reset index
                result_df = right_merged.drop(columns=['id1', 'id2', 'id_left', 'id_right'])
                result_df = right_merged.copy()

                # Merge with match_df to determine labels
                merged_df = result_df.merge(match_df, left_on=['id_left', 'id_right'], right_on=['ltable_id', 'rtable_id'], how='left', indicator=True)
                result_df['label'] = (merged_df['_merge'] == 'both').astype(int)
                result_df.to_csv(task +'_'+method+'_'+str(meta)+ '.csv',index= False)

                # Calculate metrics
                RR = 1 - result_df.shape[0] / (left_df.shape[0] * right_df.shape[0])
                RR_opt = 1 - match_df.shape[0] / (left_df.shape[0] * right_df.shape[0])
                PC = np.sum(result_df['label'] == 1) / match_df.shape[0]
                PQ = np.sum(result_df['label'] == 1) / result_df.shape[0]
                F = 2 * PC * PQ / (PC + PQ)

                # Print and save results
                print(task, classic_method_name[method])
                print(round(100 * RR, 4), round(100 * PC, 4), round(100 * PQ, 4), round(100 * F, 4))
                print()

                runtime = end_time - start_time

                with open(output_file, "a") as file:
                    file.write(f"{task} {classic_method_name[method]}\n")
                    file.write(f"{round(100 * RR, 4)} {round(100 * PC, 4)} {round(100 * PQ, 4)} {round(100 * F, 4)} {round(runtime, 2)}\n\n")


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mohammad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Comparison Propagation:   0%|          | 0/6907 [00:00<?, ?it/s]